In [0]:
# Work inside the same database
spark.sql("USE dbx_demo")

# Basic metrics
metrics = spark.sql("""
SELECT
  COUNT(*) AS trips,
  AVG(trip_distance) AS avg_distance,
  AVG(fare_amount)   AS avg_fare
FROM nyc_taxi
""")
display(metrics)

# COMMAND ----------
# Top pickup zips by trips
top_pickups = spark.sql("""
SELECT pickup_zip, COUNT(*) AS trips
FROM nyc_taxi
WHERE pickup_zip IS NOT NULL
GROUP BY pickup_zip
ORDER BY trips DESC
LIMIT 10
""")
display(top_pickups)

# COMMAND ----------
# Top dropoff zips by trips
top_dropoffs = spark.sql("""
SELECT dropoff_zip, COUNT(*) AS trips
FROM nyc_taxi
WHERE dropoff_zip IS NOT NULL
GROUP BY dropoff_zip
ORDER BY trips DESC
LIMIT 10
""")
display(top_dropoffs)

# COMMAND ----------
# Create a reusable view with per-zip stats
spark.sql("""
CREATE OR REPLACE VIEW vw_zip_stats AS
SELECT
  pickup_zip,
  COUNT(*)                      AS trips,
  AVG(fare_amount)              AS avg_fare,
  AVG(trip_distance)            AS avg_distance
FROM nyc_taxi
WHERE pickup_zip IS NOT NULL
GROUP BY pickup_zip
""")

from pyspark.sql import functions as F

display(
    spark.table("vw_zip_stats")
    .orderBy(F.col("trips").desc())
    .limit(10)
)

# COMMAND ----------
# Optional: daily rollup table (handy for dashboards)
spark.sql("""
CREATE OR REPLACE TABLE daily_basic_metrics AS
SELECT
  DATE(tpep_pickup_datetime)      AS pickup_date,
  COUNT(*)                        AS trips,
  AVG(trip_distance)              AS avg_distance,
  AVG(fare_amount)                AS avg_fare
FROM nyc_taxi
GROUP BY DATE(tpep_pickup_datetime)
ORDER BY pickup_date
""")

display(spark.table("daily_basic_metrics").limit(20))
